#PROJEK 2
#Evaluasi Beberapa Algoritma Klasifikasi Pada Dataset Immunotherapy



In [ ]:
#Menentukan metode klasifikasi yang akan dievaluasi
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('DST', DecisionTreeClassifier()))
models.append(('SVM', SVC(gamma='auto')))

#evaluate each model in turn
results = []
names = []

In [ ]:
# Menggunakan minkowski distance
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
# Menggunakan kriteria GINI Index
dst = DecisionTreeClassifier()
dst.fit(X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
# Menggunakan kernel RBF
svm = SVC(gamma='auto')
svm.fit(X,y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
#Upload Dataset Immunotherapy
import pandas as pd
import numpy as np
from google.colab import files
immunotherapy = files.upload()
for fn in immunotherapy.keys():
  print('Nama file "{name}" dengan panjang {length} bytes'.format(
      name=fn, length=len(immunotherapy[fn])
      ))

Saving Immunotherapy.xlsx to Immunotherapy (17).xlsx
Nama file "Immunotherapy.xlsx" dengan panjang 33796 bytes


# **Dataset Tanpa Normalisasi**

In [ ]:
# Menampilkan Dataset
import pandas as pd
import numpy as np
data = pd.read_excel('Immunotherapy.xlsx')
data

,sex,age,Time,Number_of_Warts,Type,Area,induration_diameter,Result_of_Treatment
0,1,22,2.25,14,3,51,50,1
1,1,15,3.00,2,3,900,70,1
2,1,16,10.50,2,1,100,25,1
3,1,27,4.50,9,3,80,30,1
4,1,20,8.00,6,1,45,8,1
...,...,...,...,...,...,...,...,...
85,1,40,5.50,8,3,69,5,1
86,1,38,7.50,8,2,56,45,1
87,1,46,11.50,4,1,91,25,0
88,1,32,12.00,9,1,43,50,0


In [ ]:
data.head()

,sex,age,Time,Number_of_Warts,Type,Area,induration_diameter,Result_of_Treatment
0,1,22,2.25,14,3,51,50,1
1,1,15,3.00,2,3,900,70,1
2,1,16,10.50,2,1,100,25,1
3,1,27,4.50,9,3,80,30,1
4,1,20,8.00,6,1,45,8,1


In [ ]:
#Pengecekan Missing Value
data.isnull().values.any()

False

In [ ]:
def cetak_rentang(df_input):
  list_fitur = df_input.columns[:-1]
  for fitur in list_fitur:
    max = df_input[fitur].max()
    min = df_input[fitur].min()
    print("Rentang fitur ", fitur, "\tadalah ", max-min)

cetak_rentang(data)

Rentang fitur  sex 	adalah  1
Rentang fitur  age 	adalah  41
Rentang fitur  Time 	adalah  11.0
Rentang fitur  Number_of_Warts 	adalah  18
Rentang fitur  Type 	adalah  2
Rentang fitur  Area 	adalah  894
Rentang fitur  induration_diameter 	adalah  68


In [ ]:
# Membuat Set Data Validasi Data Tanpa Normalisasi
from sklearn.model_selection import train_test_split
array = data.values
X = array[:,0:-1]
y = array[:,-1]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

**K-Fold Cross Validation Data Tanpa Normalisasi**

In [ ]:
#Accuracy
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

results_acc = []
names_acc = []
scoring = 'accuracy'
for name, model in models:
  kfold = KFold(n_splits=5)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results_acc.append(cv_results)
  names_acc.append(name)
  msg = "%s: %.2f%%" % (name, (cv_results.mean()*100))
  print(msg)

KNN: 80.57%
DST: 76.19%
SVM: 81.90%


Hasil akurasi terbaik: SVM

In [ ]:
#Precision
results_prec = []
names_prec = []
scoring = 'precision'
for name, model in models:
  kfold = KFold(n_splits=5)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results_prec.append(cv_results)
  names_prec.append(name)
  msg = "%s: %.2f%%" % (name, (cv_results.mean()*100))
  print(msg)

KNN: 81.81%
DST: 86.99%
SVM: 81.90%


Hasil presisi terbaik: DST

In [ ]:
#Specificity
from sklearn.metrics import confusion_matrix

results_spec = []
names_spec = []
for name, model in models:
  kfold = KFold(n_splits=5)
  model.fit(X_train, Y_train)
  predicted = model.predict(X_validation)
  matrix = confusion_matrix(Y_validation, predicted)
  TN = matrix[0][0]
  FP = matrix[0][1]
  spec_results = (TN / float(TN + FP))
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results_spec.append(spec_results)
  names_spec.append(name)
  msg = "%s: %.2f%%" % (name, (spec_results.mean()*100))
  print(msg)

KNN: 16.67%
DST: 66.67%
SVM: 0.00%


Hasil spesifisitas terbaik: DST

# **Dataset dengan Normalisasi Minmax**

In [ ]:
# Menampilkan Dataset
import pandas as pd
import numpy as np
data_minmax = pd.read_excel('Immunotherapy.xlsx')
data_minmax.head()

,sex,age,Time,Number_of_Warts,Type,Area,induration_diameter,Result_of_Treatment
0,1,22,2.25,14,3,51,50,1
1,1,15,3.00,2,3,900,70,1
2,1,16,10.50,2,1,100,25,1
3,1,27,4.50,9,3,80,30,1
4,1,20,8.00,6,1,45,8,1


In [ ]:
def minmax(df_input):
  list_fitur = df_input.columns[:-1]
  for fitur in list_fitur:
    max = df_input[fitur].max()
    min = df_input[fitur].min()
    df_input[fitur] = (df_input[fitur]-min)/(max-min)
  return df_input

data_minmax = minmax(data)

In [ ]:
cetak_rentang(data_minmax)

Rentang fitur  sex 	adalah  1.0
Rentang fitur  age 	adalah  1.0
Rentang fitur  Time 	adalah  1.0
Rentang fitur  Number_of_Warts 	adalah  1.0
Rentang fitur  Type 	adalah  1.0
Rentang fitur  Area 	adalah  1.0
Rentang fitur  induration_diameter 	adalah  1.0


In [ ]:
# Membuat Set Data Validasi Data Normalisasi Minmax
from sklearn.model_selection import train_test_split
array_minmax = data_minmax.values
Xm = array_minmax[:,0:-1]
ym = array_minmax[:,-1]
Xm_train, Xm_validation, Ym_train, Ym_validation = train_test_split(Xm, ym, test_size=0.20, random_state=1, shuffle=True)

**K-Fold Cross Validation Data Normalisasi Minmax**

In [ ]:
#Accuracy
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

results_acc_m = []
names_acc_m = []
scoring_m = 'accuracy'
for name, model in models:
  kfold_m = KFold(n_splits=5)
  cv_results_m = cross_val_score(model, Xm_train, Ym_train, cv=kfold_m, scoring=scoring_m)
  results_acc_m.append(cv_results_m)
  names_acc_m.append(name)
  msg_m = "%s: %.2f%%" % (name, (cv_results_m.mean()*100))
  print(msg_m)

KNN: 79.24%
DST: 80.29%
SVM: 81.90%


Hasil akurasi terbaik : SVM

In [ ]:
#Precision
results_prec_m = []
names_prec_m = []
scoring_m = 'precision'
for name, model in models:
  kfold_m = KFold(n_splits=5)
  cv_results_m = cross_val_score(model, Xm_train, Ym_train, cv=kfold_m, scoring=scoring_m)
  results_prec_m.append(cv_results_m)
  names_prec_m.append(name)
  msg_m = "%s: %.2f%%" % (name, (cv_results_m.mean()*100))
  print(msg_m)

KNN: 81.43%
DST: 87.44%
SVM: 81.90%


Hasil presisi terbaik : DST

In [ ]:
#Specificity
from sklearn.metrics import confusion_matrix

results_spec_m = []
names_spec_m = []
for name, model in models:
  kfold_m = KFold(n_splits=5)
  model.fit(Xm_train, Ym_train)
  predicted = model.predict(Xm_validation)
  matrix = confusion_matrix(Ym_validation, predicted)
  TN = matrix[0][0]
  FP = matrix[0][1]
  spec_results_m = (TN / float(TN + FP))
  cv_results_m = cross_val_score(model, Xm_train, Ym_train, cv=kfold_m, scoring=scoring_m)
  results_spec_m.append(spec_results_m)
  names_spec_m.append(name)
  msg_m = "%s: %.2f%%" % (name, (spec_results_m.mean()*100))
  print(msg_m)

KNN: 16.67%
DST: 83.33%
SVM: 0.00%


Hasil spesifisitas terbaik : DST

# **Evaluasi Prediktor Menggunakan SVM dengan Data Normalisasi**

### **a) SVM dengan Kernel RBF**

In [ ]:
# Buat Prediksi SVM Kernel RBF
svm = SVC(gamma='auto')

In [ ]:
svm.fit(Xm_train, Ym_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
predictions_svm = svm.predict(Xm_validation)

In [ ]:
ym_pred_svm = svm.predict(Xm_validation)
ym_pred_svm

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [ ]:
# Evaluasi Prediksi
from sklearn.datasets import make_classification
Xm, ym = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Ym_validation, ym_pred_svm))
print(classification_report(Ym_validation, ym_pred_svm))

[[ 0  6]
 [ 0 12]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         6
         1.0       0.67      1.00      0.80        12

    accuracy                           0.67        18
   macro avg       0.33      0.50      0.40        18
weighted avg       0.44      0.67      0.53        18



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
akurasi_rbf = metrics.accuracy_score(Ym_validation,ym_pred_svm)
print('Accuracy \t: %.2f%%'% (akurasi_rbf*100))

presisi_rbf = metrics.accuracy_score(Ym_validation,ym_pred_svm)
print('Precision \t: %.2f%%'% (presisi_rbf*100))

specificity_rbf = TN/(TN+FP)
print('Specificity \t: %.2f%%'% (specificity_rbf*100))

Accuracy 	: 66.67%
Precision 	: 66.67%
Specificity 	: 0.00%


### **b) SVM dengan Kernel Linear**

In [ ]:
# Buat Prediksi SVM Kernel Linear
from sklearn.svm import LinearSVC
svm_linear = LinearSVC()
svm_linear.fit(Xm_train, Ym_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
predictions_svm_linear = svm_linear.predict(Xm_validation)

In [ ]:
ym_pred_svmlinear = svm_linear.predict(X_validation)
ym_pred_svmlinear

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [ ]:
# Evaluasi Prediksi
from sklearn.datasets import make_classification
Xm, ym = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Ym_validation, ym_pred_svmlinear))
print(classification_report(Ym_validation, ym_pred_svmlinear))

[[ 6  0]
 [12  0]]
              precision    recall  f1-score   support

         0.0       0.33      1.00      0.50         6
         1.0       0.00      0.00      0.00        12

    accuracy                           0.33        18
   macro avg       0.17      0.50      0.25        18
weighted avg       0.11      0.33      0.17        18



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
akurasi_linear = metrics.accuracy_score(Ym_validation,ym_pred_svmlinear)
print('Accuracy \t: %.2f%%'% (akurasi_linear*100))

presisi_linear = metrics.accuracy_score(Ym_validation,ym_pred_svmlinear)
print('Precision \t: %.2f%%'% (presisi_linear*100))

specificity_lin = TN/(TN+FP)
print('Specificity \t: %.2f%%'% (specificity_lin*100))

Accuracy 	: 33.33%
Precision 	: 33.33%
Specificity 	: 0.00%


### **c) SVM dengan Kernel Polynomial**

In [ ]:
# Buat Prediksi SVM Kernel Polynomial
svm_poly = SVC(kernel='poly', degree=8)
svm_poly.fit(Xm_train, Ym_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=8, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
predictions_svm_poly = svm_poly.predict(Xm_validation)

In [ ]:
ym_pred_svmpoly = svm_poly.predict(Xm_validation)
ym_pred_svmpoly

array([1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       1.])

In [ ]:
# Evaluasi Prediksi
from sklearn.datasets import make_classification
Xm, ym = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Ym_validation, ym_pred_svmpoly))
print(classification_report(Ym_validation, ym_pred_svmpoly))

[[1 5]
 [3 9]]
              precision    recall  f1-score   support

         0.0       0.25      0.17      0.20         6
         1.0       0.64      0.75      0.69        12

    accuracy                           0.56        18
   macro avg       0.45      0.46      0.45        18
weighted avg       0.51      0.56      0.53        18



In [ ]:
akurasi_poly = metrics.accuracy_score(Ym_validation,ym_pred_svmpoly)
print('Accuracy \t: %.2f%%'% (akurasi_poly*100))

presisi_poly = metrics.accuracy_score(Ym_validation,ym_pred_svmpoly)
print('Precision \t: %.2f%%'% (presisi_poly*100))

specificity_poly = TN/(TN+FP)
print('Specificity \t: %.2f%%'% (specificity_poly*100))

Accuracy 	: 55.56%
Precision 	: 55.56%
Specificity 	: 0.00%
